In [1]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [1]:
import subprocess
from pathlib import Path
from collections import Counter

In [12]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8008/v1',
    '--api-key', 'xxx',
    '--model-name', 'Qwen/Qwen3-4B',
    '--chat-template-model-id', 'Qwen/Qwen3-4B',
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '500',
    '--prompt', 'Write a story about a princess. '
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)

Running: python3 main.py --api-key xxx --model-name Qwen/Qwen3-4B --chat-template-model-id Qwen/Qwen3-4B --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 500 --prompt Write a story about a princess. 
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
--- Generation start ---

--- Generated Output (stream) ---
<|im_start|>user
Write a story about a princess. <|im_end|>
<|im_start|>assistant
<think>

</think>


<|im_start|>user
Write a story about a princess. <|im_end|>
<|im_start|>assistant
<think>

</think>

**The Princess of the Silver
<|im_start|>user
Write a story about a princess. <|im_end|>
<|im_start|>assistant
<think>

</think>

**The Princess of the Silver Moon**

In a land far beyond the reach of ordinary maps, where the sky gl
<|im_start|>user
Write a story about a princess. <|im_end|>
<|im_start|>assistant

CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--model-name', 'Qwen/Qwen3-4B', '--chat-template-model-id', 'Qwen/Qwen3-4B', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json', '--max-new-tokens', '500', '--prompt', 'Write a story about a princess. '], returncode=0)

In [6]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

THREADS  = 20
MAX_PROMPTS = 100

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8008/v1',
    '--api-key', 'xxx',
    #'--model-name', 'unsloth/Llama-3.2-3B-Instruct',
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '200'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 20 --max-prompts 100 --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 200
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   0%|          | 99/177477 [00:00<00:04, 43362.17prompt/s]
Preparing to process 100 new prompts in this run.
BANNED (prompt_idx=12, type=slop_phrase, phrase='curiosity')                
BANNED (prompt_idx=11, type=ngram, ngram='dark lord')                      
BANNED (prompt_idx=15, type=slop_phrase, phrase='stark')                   
BANNED (prompt_idx=13, type=slop_phrase, phrase='footsteps')               
BANNED (prompt_idx=10, type=slop_phrase, phrase='couldn't help but feel')  
B

CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '20', '--max-prompts', '100', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json', '--max-new-tokens', '200'], returncode=0)